In [14]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.metrics import confusion_matrix, classification_report

from rtbm import RTBM, minimizer

import rtbm.layers as layers
import rtbm.model as mdl

import warnings
warnings.filterwarnings('ignore')

from rtbm.costfunctions import mse, crossentropy


In [15]:
# Load MNIST dataset
MNIST_train = pd.read_csv('~/data/mnist_train.csv', delimiter=",",header=None).values
MNIST_test  = pd.read_csv('~/data/mnist_test.csv', delimiter=",",header=None).values

# Prepare data (normalized onto [0,1])
Y_train = MNIST_train[0:10000,0]
X_train = MNIST_train[0:10000,1:]/255.0

Y_test = MNIST_test[:,0]
X_test = MNIST_test[:,1:]/255.0

In [ ]:
# Visualize individual pics
i=10
print(Y_train[i])
I=np.reshape(X_train[i], (28,28))
plt.imshow(I, interpolation='nearest',  cmap='gray_r')
plt.show()

# Logistic regression baseline

In [ ]:
# Logistic regression baseline
logreg = linear_model.LogisticRegression(multi_class='multinomial',solver='lbfgs')

logreg.fit(X_train,Y_train)


In [ ]:
# On train set

P=logreg.predict(X_train)

print(classification_report(Y_train,P))
print(confusion_matrix(Y_train, P))


In [ ]:
# On test set
P=logreg.predict(X_test)

print(classification_report(Y_test,P))
print(confusion_matrix(Y_test, P))


# Linear regression base line

In [10]:
linreg = linear_model.LinearRegression()

linreg.fit(X_train,Y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [13]:
# On train set

P=np.round(linreg.predict(X_train))

print(classification_report(Y_train,P))
print(confusion_matrix(Y_train, P))

             precision    recall  f1-score   support

       -2.0       0.00      0.00      0.00         0
       -1.0       0.00      0.00      0.00         0
        0.0       0.81      0.24      0.37      1001
        1.0       0.38      0.25      0.30      1127
        2.0       0.20      0.26      0.23       991
        3.0       0.27      0.33      0.30      1032
        4.0       0.15      0.20      0.17       980
        5.0       0.17      0.31      0.22       863
        6.0       0.22      0.29      0.25      1014
        7.0       0.31      0.31      0.31      1070
        8.0       0.15      0.11      0.13       944
        9.0       0.57      0.15      0.24       978
       10.0       0.00      0.00      0.00         0
       11.0       0.00      0.00      0.00         0
       12.0       0.00      0.00      0.00         0

avg / total       0.33      0.25      0.25     10000

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0

In [ ]:
# On test set
P=linreg.predict(X_test)

print(classification_report(Y_test,P))
print(confusion_matrix(Y_test, P))


# Linear regression via CMA

In [ ]:
M = mdl.Model()
M.add(layers.Linear(784,1,paramBound=2))

minim = minimizer.CMA(False)
sol=minim.train(mse(), M, np.transpose(X_train), np.transpose(Y_train), maxiter=1000)

In [ ]:
# On train set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_train)))))


print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

In [ ]:
# On test set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_test)))))

print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))

# Linear regression via SGD

In [6]:
M = mdl.Model()
M.add(layers.Linear(784,1))

minim = minimizer.SGD()
sol=minim.train(mse(), M, np.transpose(X_train), np.transpose(Y_train), lr=0.01, maxiter=100000)

(0, ' cost: ', 52.405416241751951)
(100, ' cost: ', 4.5315394680893437)
(200, ' cost: ', 3.5806252308132587)
(300, ' cost: ', 3.189365917081473)
(400, ' cost: ', 2.9570966901213569)
(500, ' cost: ', 2.7930213232545089)
(600, ' cost: ', 2.6667520146372561)
(700, ' cost: ', 2.5649458776182308)
(800, ' cost: ', 2.4804390836876524)
(900, ' cost: ', 2.4088418180500577)
(1000, ' cost: ', 2.347231460767865)
(1100, ' cost: ', 2.2935541663190713)
(1200, ' cost: ', 2.2463104865445929)
(1300, ' cost: ', 2.2043733120360405)
(1400, ' cost: ', 2.1668751327206888)
(1500, ' cost: ', 2.1331348375483694)
(1600, ' cost: ', 2.1026084667308256)
(1700, ' cost: ', 2.074855123049995)
(1800, ' cost: ', 2.0495128036025694)
(1900, ' cost: ', 2.0262809021162433)
(2000, ' cost: ', 2.0049073011254444)
(2100, ' cost: ', 1.9851786870087522)
(2200, ' cost: ', 1.9669131697744962)
(2300, ' cost: ', 1.9499545788465242)
(2400, ' cost: ', 1.9341679966209819)
(2500, ' cost: ', 1.9194362193686429)
(2600, ' cost: ', 1.9056569

(21400, ' cost: ', 1.5361898991228717)
(21500, ' cost: ', 1.5358952941318826)
(21600, ' cost: ', 1.5356034589174341)
(21700, ' cost: ', 1.5353143563930858)
(21800, ' cost: ', 1.5350279501180695)
(21900, ' cost: ', 1.534744204283377)
(22000, ' cost: ', 1.5344630836982076)
(22100, ' cost: ', 1.5341845537767591)
(22200, ' cost: ', 1.5339085805253565)
(22300, ' cost: ', 1.5336351305299072)
(22400, ' cost: ', 1.5333641709436718)
(22500, ' cost: ', 1.533095669475345)
(22600, ' cost: ', 1.5328295943774313)
(22700, ' cost: ', 1.5325659144349149)
(22800, ' cost: ', 1.5323045989542068)
(22900, ' cost: ', 1.5320456177523694)
(23000, ' cost: ', 1.5317889411466037)
(23100, ' cost: ', 1.5315345399439952)
(23200, ' cost: ', 1.5312823854315127)
(23300, ' cost: ', 1.5310324493662462)
(23400, ' cost: ', 1.5307847039658871)
(23500, ' cost: ', 1.5305391218994338)
(23600, ' cost: ', 1.5302956762781224)
(23700, ' cost: ', 1.5300543406465759)
(23800, ' cost: ', 1.5298150889741635)
(23900, ' cost: ', 1.529577

(42500, ' cost: ', 1.5054706810123737)
(42600, ' cost: ', 1.5054007307550767)
(42700, ' cost: ', 1.5053311300284571)
(42800, ' cost: ', 1.5052618762559624)
(42900, ' cost: ', 1.5051929668853488)
(43000, ' cost: ', 1.5051243993883987)
(43100, ' cost: ', 1.5050561712606485)
(43200, ' cost: ', 1.5049882800211167)
(43300, ' cost: ', 1.5049207232120363)
(43400, ' cost: ', 1.5048534983985922)
(43500, ' cost: ', 1.5047866031686585)
(43600, ' cost: ', 1.5047200351325438)
(43700, ' cost: ', 1.5046537919227374)
(43800, ' cost: ', 1.5045878711936587)
(43900, ' cost: ', 1.504522270621411)
(44000, ' cost: ', 1.5044569879035377)
(44100, ' cost: ', 1.5043920207587826)
(44200, ' cost: ', 1.5043273669268529)
(44300, ' cost: ', 1.5042630241681845)
(44400, ' cost: ', 1.5041989902637127)
(44500, ' cost: ', 1.5041352630146421)
(44600, ' cost: ', 1.5040718402422251)
(44700, ' cost: ', 1.5040087197875358)
(44800, ' cost: ', 1.5039458995112556)
(44900, ' cost: ', 1.5038833772934541)
(45000, ' cost: ', 1.50382

(63600, ' cost: ', 1.4957979334308249)
(63700, ' cost: ', 1.4957679210413981)
(63800, ' cost: ', 1.4957380047537279)
(63900, ' cost: ', 1.4957081840798256)
(64000, ' cost: ', 1.495678458534921)
(64100, ' cost: ', 1.4956488276374351)
(64200, ' cost: ', 1.4956192909089574)
(64300, ' cost: ', 1.4955898478742182)
(64400, ' cost: ', 1.4955604980610653)
(64500, ' cost: ', 1.4955312410004371)
(64600, ' cost: ', 1.4955020762263402)
(64700, ' cost: ', 1.4954730032758228)
(64800, ' cost: ', 1.4954440216889529)
(64900, ' cost: ', 1.495415131008792)
(65000, ' cost: ', 1.4953863307813728)
(65100, ' cost: ', 1.4953576205556762)
(65200, ' cost: ', 1.4953289998836059)
(65300, ' cost: ', 1.4953004683199671)
(65400, ' cost: ', 1.4952720254224436)
(65500, ' cost: ', 1.4952436707515735)
(65600, ' cost: ', 1.4952154038707299)
(65700, ' cost: ', 1.4951872243460949)
(65800, ' cost: ', 1.4951591317466408)
(65900, ' cost: ', 1.4951311256441058)
(66000, ' cost: ', 1.4951032056129741)
(66100, ' cost: ', 1.495075

(84700, ' cost: ', 1.4910348397283502)
(84800, ' cost: ', 1.4910177765624757)
(84900, ' cost: ', 1.4910007513838428)
(85000, ' cost: ', 1.49098376404818)
(85100, ' cost: ', 1.4909668144119446)
(85200, ' cost: ', 1.4909499023323158)
(85300, ' cost: ', 1.490933027667195)
(85400, ' cost: ', 1.490916190275196)
(85500, ' cost: ', 1.490899390015644)
(85600, ' cost: ', 1.4908826267485711)
(85700, ' cost: ', 1.4908659003347107)
(85800, ' cost: ', 1.4908492106354927)
(85900, ' cost: ', 1.4908325575130419)
(86000, ' cost: ', 1.4908159408301715)
(86100, ' cost: ', 1.49079936045038)
(86200, ' cost: ', 1.4907828162378454)
(86300, ' cost: ', 1.4907663080574227)
(86400, ' cost: ', 1.4907498357746407)
(86500, ' cost: ', 1.4907333992556948)
(86600, ' cost: ', 1.4907169983674449)
(86700, ' cost: ', 1.4907006329774122)
(86800, ' cost: ', 1.4906843029537729)
(86900, ' cost: ', 1.4906680081653565)
(87000, ' cost: ', 1.4906517484816402)
(87100, ' cost: ', 1.4906355237727456)
(87200, ' cost: ', 1.49061933390

In [9]:
# On train set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_train)))))


print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

             precision    recall  f1-score   support

        0.0       0.81      0.22      0.34      1001
        1.0       0.34      0.26      0.29      1127
        2.0       0.21      0.27      0.24       991
        3.0       0.27      0.31      0.29      1032
        4.0       0.14      0.20      0.16       980
        5.0       0.17      0.31      0.22       863
        6.0       0.22      0.29      0.25      1014
        7.0       0.29      0.30      0.29      1070
        8.0       0.15      0.11      0.12       944
        9.0       0.57      0.15      0.24       978
       10.0       0.00      0.00      0.00         0
       11.0       0.00      0.00      0.00         0
       12.0       0.00      0.00      0.00         0

avg / total       0.32      0.24      0.25     10000

[[216 386 235 100  35  21   7   1   0   0   0   0   0]
 [  5 288 521 200  82  25   4   2   0   0   0   0   0]
 [ 34 113 269 294 183  75  17   6   0   0   0   0   0]
 [  9  52 192 325 288 105  26  23   6

In [8]:
# On test set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_test)))))

print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))


             precision    recall  f1-score   support

        0.0       0.78      0.19      0.31       980
        1.0       0.33      0.25      0.29      1135
        2.0       0.17      0.21      0.19      1032
        3.0       0.25      0.30      0.27      1010
        4.0       0.15      0.20      0.17       982
        5.0       0.18      0.31      0.23       892
        6.0       0.17      0.24      0.19       958
        7.0       0.25      0.27      0.26      1028
        8.0       0.16      0.12      0.14       974
        9.0       0.45      0.10      0.16      1009
       10.0       0.00      0.00      0.00         0
       11.0       0.00      0.00      0.00         0
       12.0       0.00      0.00      0.00         0
       14.0       0.00      0.00      0.00         0

avg / total       0.29      0.22      0.22     10000

[[190 356 235 128  43  20   5   3   0   0   0   0   0   0]
 [  3 285 539 197  77  27   7   0   0   0   0   0   0   0]
 [ 36 143 218 288 212  99  26  

# E(h|v) via SGD

In [16]:
M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(784,10))
M.add(layers.DiagExpectationUnitLayer(10,1))

minim = minimizer.SGD()
sol=minim.train(mse(), M, np.transpose(X_train), np.transpose(Y_train), lr=0.1, maxiter=1000)

(0, ' cost: ', (14.32653127447769+0j))


LinAlgError: Matrix is not positive definite

In [ ]:
# On train set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_train)))))


print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

In [ ]:
# On test set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_test)))))

print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))



# E(h|v) via CMA

In [ ]:
M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(784,1))

minim = minimizer.CMA(True)
sol=minim.train(mse(), M, np.transpose(X_train), np.transpose(Y_train), maxiter=1000)

In [ ]:
# On train set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_train)))))


print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

In [ ]:
# On test set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_test)))))

print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))
